<a href="https://colab.research.google.com/github/gtmray/NLP/blob/master/NLP_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df1 = pd.read_csv('https://raw.githubusercontent.com/gtmray/imdb/master/imdb_train.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/gtmray/imdb/master/imdb_test.csv')
df = df1.append(df2)
df_decreased = df.iloc[:50000, :]
print(df_decreased.shape)
# paragraph = "My 123 name is Rewan Gautam Rewan. I come from the heart of purbanchal, Itahari. It is biggest city of Sunsari district. I am the person you will not predict even with neural networks."
# sentence = nltk.sent_tokenize(paragraph)
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


(50000, 2)


In [ ]:
#Converting sentences to cleaned sentences with 
# - lowercase
# - taking only alphabets
# - lemmatization

import re
stop = stopwords.words('english')
cleaned = []
sentence = df_decreased['text']
for i in sentence:
  i = re.sub('[^a-zA-Z]', ' ', i)
  i = i.lower()
  words = nltk.word_tokenize(i)
  temp = [stemmer.stem(word) for word in words if word not in stop]
  cleaned.append(" ".join(temp))

In [ ]:
#Bag of Words Model
#  - rows = sentences
#  - columns = words

cleaned = df_decreased['text']
from sklearn.feature_extraction.text import CountVectorizer #Count of words in each sentence
cv = CountVectorizer(max_features=5000, stop_words='english')
X = cv.fit_transform(cleaned).toarray()

#Bag of words is not good as tfidf because it has no semantic meaning but perform well in sentiment analysis

In [ ]:
'''
Term Frequency(tf) = No. of repeated word in sentence / No. of words in sentence
Inverse Document Frequency(idf) = log(No. of sentences/No. of sentence containing words)
tfidf = tf*idf
'''

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
X_tfidf = tfidf.fit_transform(cleaned).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df_decreased.label, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))


[[4263  701]
 [ 754 4282]]
0.8545
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      4964
           1       0.86      0.85      0.85      5036

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



<h1>*******Using Word2Vec*******<h1>

In [ ]:
######## Word2Vec ##########
'''
In both bag of words and tfidf, semantic information  is not stored. 
TF-IDF gives importance to uncommon words.
Chance of over fitting.

- The solution is Word2Vec. Here, each word is represented as a vector of 32 or
more dimension instead of a single number.
- Here the semantic information and relation between different words is also
preserved
'''

'\nIn both bag of words and tfidf, semantic information  is not stored. \nTF-IDF gives importance to uncommon words.\nChance of over fitting.\n\n- The solution is Word2Vec. Here, each word is represented as a vector of 32 or\nmore dimension instead of a single number.\n- Here the semantic information and relation between different words is also\npreserved\n'

In [ ]:
import re
from gensim.models import Word2Vec
paragraph = "My 123 name is Rewan Gautam Rewan. I come from the heart of purbanchal, Itahari. It is biggest city of Sunsari district. I am the person you will not predict even with neural networks."

In [ ]:
#Data proprocessing

text = re.sub(r'\[[0-9]*\]', ' ', paragraph)
text = re.sub(r'\s+', ' ', text)
text = text.lower()
text = re.sub(r'\d', ' ', text)
text = re.sub(r'\s+', ' ', text)

sentences = nltk.sent_tokenize(text)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

for i in range(len(sentences)):
  sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]

In [ ]:
#Training Word2Vec model

model = Word2Vec(sentences, min_count=1)
words = model.wv.vocab
vector = model.wv['rewan']
similar = model.wv.most_similar('rewan')
print(similar)

[('.', 0.025892771780490875), ('town', 0.016688115894794464), ('kalabanjar', -0.005918759852647781), ('district', -0.010986454784870148), ('engineer', -0.032506052404642105), ('gautam', -0.042173877358436584), ('born', -0.0720982477068901), ('name', -0.08349581807851791), ('biggest', -0.09935536980628967), ('sunsari', -0.11096066981554031)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<h1>Fake news classifier</h1>

In [ ]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle 
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list
# ! kaggle competitions download -c fake-news
# !unzip train.csv.zip
# !unzip test.csv.zip

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')
print(df.columns)
df = df.dropna()
df.drop('id', axis=1, inplace=True)
df['text_new'] = df['title']

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')


In [ ]:
cleaned = df['text_new']

from sklearn.feature_extraction.text import CountVectorizer #Count of words in each sentence
cv = CountVectorizer(max_features=5000, stop_words='english')
X = cv.fit_transform(cleaned).toarray()

# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
# X_tfidf = tfidf.fit_transform(cleaned).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, df.label, test_size=0.2, random_state=42)
#print(X_test.shape)
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[1850  172]
 [ 232 1403]]
0.8895269346458846
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      2022
           1       0.89      0.86      0.87      1635

    accuracy                           0.89      3657
   macro avg       0.89      0.89      0.89      3657
weighted avg       0.89      0.89      0.89      3657



<h1>Fake News Classifier using deep learning with LSTM</h1>


In [1]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle 
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list
# ! kaggle competitions download -c fake-news
# !unzip train.csv.zip
# !unzip test.csv.zip

Saving kaggle.json to kaggle.json
ref                                                               title                                             size  lastUpdated          downloadCount  
----------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
Cornell-University/arxiv                                          arXiv Dataset                                      2GB  2020-05-06 23:18:35            359  
jeffreybraun/chipotle-locations                                   Chipotle Locations                               124KB  2020-07-28 20:20:41            140  
christianlillelund/passenger-list-for-the-estonia-ferry-disaster  MS Estonia Disaster Passenger List                14KB  2020-07-26 15:40:17             67  
jeffreybraun/chopped-10-years-of-episode-data                     Chopped: 10+ Years of Episode Data               305KB  2020-07-30 15:19:41             37  
ahsen1330/us

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/train.csv')
df = df.dropna()
X = df.drop('label', axis=1)
y = df['label']

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#Preprocessing

text = X.copy()
text.reset_index(inplace=True)

corpus = []
ps = PorterStemmer()

for i in range(0, len(text)):
  cleaning = re.sub('[^a-zA-Z]', ' ', text['title'][i])
  cleaning = cleaning.lower()
  cleaning = cleaning.split()

  cleaning = [ps.stem(word) for word in cleaning if not word in stopwords.words('english')]
  cleaning = " ".join(cleaning)
  corpus.append(cleaning)

In [5]:
voc_size = 5000 #Number of words for the one hot encoding
sent_length = 20 #Max length for padding
embedding_vector_features = 40 #Number of vector features for embedding

#One hot encoding
onehot_repr = [one_hot(sentence, voc_size) for sentence in corpus]

#Padding
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [16]:
#Model

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.4))

model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(LSTM(100))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(1, activation='sigmoid'))

model.summary()

#Compile model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 100)           56400     
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 100)           400       
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
batch_normalization_3 (Batch (None, 20, 100)          

In [17]:
#Converting to numpy array

X_final = np.array(embedded_docs)

y_final = np.array(y)

In [18]:
#Splitting dataset to training and testing 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=77)

#Model training

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=100)

Epoch 1/20
147/147 [==============================] - 23s 153ms/step - loss: 0.2793 - accuracy: 0.8820 - val_loss: 0.5827 - val_accuracy: 0.5649
Epoch 2/20
147/147 [==============================] - 21s 142ms/step - loss: 0.1584 - accuracy: 0.9380 - val_loss: 0.8199 - val_accuracy: 0.5346
Epoch 3/20
147/147 [==============================] - 21s 145ms/step - loss: 0.1211 - accuracy: 0.9532 - val_loss: 0.2060 - val_accuracy: 0.9196
Epoch 4/20
147/147 [==============================] - 21s 144ms/step - loss: 0.1037 - accuracy: 0.9624 - val_loss: 0.2149 - val_accuracy: 0.9141
Epoch 5/20
147/147 [==============================] - 21s 145ms/step - loss: 0.0896 - accuracy: 0.9677 - val_loss: 0.2902 - val_accuracy: 0.9128
Epoch 6/20
147/147 [==============================] - 21s 144ms/step - loss: 0.0721 - accuracy: 0.9739 - val_loss: 0.3156 - val_accuracy: 0.9196
Epoch 7/20
147/147 [==============================] - 21s 146ms/step - loss: 0.0689 - accuracy: 0.9758 - val_loss: 0.3427 - val_ac

In [19]:
#Model performance

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[1886  180]
 [ 149 1442]]
0.910035548263604


In [71]:
def preprocessing_and_all(dataframe_dir):
  df_test = pd.read_csv(dataframe_dir)
  df_temp = df_test.copy()
  df_temp['text'] = df_temp['text'].str[:10]
  df_temp['title'] = df_temp['title'].fillna(df_temp['text'])
  df_temp.reset_index(inplace=True)

  corpus = []
  ps = PorterStemmer()

  for i in range(0, len(df_temp)):
    cleaning = re.sub('[^a-zA-Z]', ' ', df_temp['title'][i])
    cleaning = cleaning.lower()
    cleaning = cleaning.split()

    cleaning = [ps.stem(word) for word in cleaning if not word in stopwords.words('english')]
    cleaning = " ".join(cleaning)
    corpus.append(cleaning)
  
  voc_size = 5000 #Number of words for the one hot encoding
  sent_length = 20 #Max length for padding
  embedding_vector_features = 40 #Number of vector features for embedding

  #One hot encoding
  onehot_repr = [one_hot(sentence, voc_size) for sentence in corpus]

  #Padding
  embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
  
  X = np.array(embedded_docs)
  y_pred_real = model.predict_classes(X)  
  df_temp['label'] = y_pred_real
  df_to_submit = df_temp[['id', 'label']]
  return df_to_submit

df_sum = preprocessing_and_all('/content/test.csv')
print(df_sum.head())
print(df_sum.info())
df_sum.to_csv('/content/final.csv', index=False)
!kaggle competitions submit fake-news -f /content/final.csv -m "My submission message"

      id  label
0  20800      0
1  20801      1
2  20802      0
3  20803      0
4  20804      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      5200 non-null   int64
 1   label   5200 non-null   int32
dtypes: int32(1), int64(1)
memory usage: 61.1 KB
None
100% 40.6k/40.6k [00:06<00:00, 6.32kB/s]
Successfully submitted to Fake News